<a href="https://colab.research.google.com/github/AadeshVarude/Semantic_segmentation/blob/main/Semantic_segmentation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.utils.data as data
import torchvision
from torchvision import transforms
import torch.optim as optim
import matplotlib.pyplot as plt
import cv2 
import os
from tqdm import tqdm
import matplotlib.pyplot as plt

In [2]:
# torch.cuda.set_device(0)

In [3]:
from google.colab import drive
drive.mount('/content/gdrive');

Mounted at /content/gdrive


In [4]:
!pip install pytorch_lightning -qq
!pip install --upgrade wandb -qq

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 800.3/800.3 KB 27.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 125.4/125.4 KB 18.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 512.4/512.4 KB 47.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 50.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 177.4/177.4 KB 24.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 184.0/184.0 KB 25.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 KB 8.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 140.6/140.6 KB 20.0 MB/s eta 0:00:00


In [5]:
%matplotlib inline
import os
from argparse import ArgumentParser
import torch
import torch.nn as nn
from torch.nn import functional as F
from torch.utils.data import DataLoader
from torchvision.datasets import MNIST
import torchvision.transforms as transforms
import torchvision
import matplotlib.pyplot as plt
import pytorch_lightning as pl
from torch.utils import data

# torch.cuda.set_device(0)

In [6]:
class DoubleConv(nn.Module) :
  '''
  Double Convolution and BN and ReLU
  (3x3 conv -> BN -> ReLU) ** 2
  '''
  def __init__(self, in_ch, out_ch) :
    super().__init__()
    self.net = nn.Sequential(
      nn.Conv2d(in_ch, out_ch, kernel_size = 3, padding = 1),
      nn.BatchNorm2d(out_ch),
      nn.ReLU(inplace = True),
      nn.Conv2d(out_ch, out_ch, kernel_size = 3, padding = 1),
      nn.BatchNorm2d(out_ch),
      nn.ReLU(inplace = True)
    )
      
  def forward(self, x) :
    return self.net(x)

class Down(nn.Module) :
  '''
  Combination of MaxPool2d and DoubleConv in series
  '''
  def __init__(self, in_ch, out_ch) : 
    super().__init__()
    self.net = nn.Sequential(
        nn.MaxPool2d(kernel_size = 2, stride = 2),
        DoubleConv(in_ch, out_ch)
    )
      
  def forward(self, x) :
    return self.net(x)
    
class Up(nn.Module) :
  '''
  Upsampling (by either bilinear interpolation or transpose convolutions) 
  followed by concatenation of feature map from contracting path, 
  followed by double 3x3 convolution.
  '''
  def __init__(self, in_ch, out_ch, bilinear = False) :
    super().__init__()
    self.upsample = None
    if bilinear :
      self.upsample = nn.Upsample(scale_factor = 2, mode = 'bilinear', align_corners = True)
    else : 
      self.upsample = nn.ConvTranspose2d(in_ch, in_ch // 2, kernel_size = 2, stride = 2)
        
    self.conv = DoubleConv(in_ch, out_ch)
      
  def forward(self, x1, x2) :
    x1 = self.upsample(x1)
    
    # Pad x1 to the size of x2
    diff_h = x2.shape[2] - x1.shape[2]
    diff_w = x2.shape[3] - x1.shape[3]
    
    x1 = F.pad(x1, [diff_w // 2, diff_w - diff_w // 2, diff_h // 2, diff_h - diff_h // 2])
    
    # Concatenate along the channels axis
    x = torch.cat([x2, x1], dim = 1)
    return self.conv(x)

In [7]:
class UNet(nn.Module) :
  def __init__(self, num_classes = 19, bilinear = False) :
    super().__init__()
    self.bilinear = bilinear
    self.num_classes = num_classes
    self.layer1 = DoubleConv(3, 64)
    self.layer2 = Down(64, 128)
    self.layer3 = Down(128, 256)
    self.layer4 = Down(256, 512)
    self.layer5 = Down(512, 1024)
    
    self.layer6 = Up(1024, 512, bilinear = self.bilinear)
    self.layer7 = Up(512, 256, bilinear = self.bilinear)
    self.layer8 = Up(256, 128, bilinear = self.bilinear)
    self.layer9 = Up(128, 64, bilinear = self.bilinear)
    
    self.layer10 = nn.Conv2d(64, self.num_classes, kernel_size = 1)
      
  def forward(self, x) :
    x1 = self.layer1(x)
    x2 = self.layer2(x1)
    x3 = self.layer3(x2)
    x4 = self.layer4(x3)
    x5 = self.layer5(x4)
    
    x6 = self.layer6(x5, x4)
    x6 = self.layer7(x6, x3)
    x6 = self.layer8(x6, x2)
    x6 = self.layer9(x6, x1)
    
    return self.layer10(x6)

In [8]:
class UnNormalize(object):
  def __init__(self, mean, std):
    self.mean = mean
    self.std = std

  def __call__(self, tensor):
    """
    Args:
        tensor (Tensor): Tensor image of size (C, H, W) to be normalized.
    Returns:
        Tensor: Normalized image.
    """
    for t, m, s in zip(tensor, self.mean, self.std):
      t.mul_(s).add_(m)
      # The normalize code -> t.sub_(m).div_(s)
    return tensor
class sem_data_creator(data.Dataset):
  def __init__(self, split = 'test', transform = None):
    self.void_labels = [0, 1, 2, 3, 4, 5, 6, 9, 10, 14, 15, 16, 18, 29, 30, -1]
    self.valid_labels = [7, 8, 11, 12, 13, 17, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 31, 32, 33]
    self.ignore_index = 250
    self.class_map = dict(zip(self.valid_labels, range(19)))
    self.split = split
    self.img_path = '/content/gdrive/MyDrive/data_semantics/testing/image_2/'
    self.mask_path = None
    if self.split == 'train':
      self.img_path = '/content/gdrive/MyDrive/data_semantics/training/image_2/'    
      self.mask_path = '/content/gdrive/MyDrive/data_semantics/training/semantic/'
    self.transform = transform
    
    self.img_list = self.get_filenames(self.img_path)
    self.mask_list = None
    if self.split == 'train':
      self.mask_list = self.get_filenames(self.mask_path)
      
  def __len__(self):
    return(len(self.img_list))
  
  def __getitem__(self, idx):
    img = cv2.imread(self.img_list[idx])
    img = cv2.resize(img, (1242, 376))
    mask = None
    if self.split == 'train':
      mask = cv2.imread(self.mask_list[idx], cv2.IMREAD_GRAYSCALE)
      mask = cv2.resize(mask, (1242, 376))
      mask = self.encode_segmap(mask)
      assert(mask.shape == (376, 1242))
    
    if self.transform:
      img = self.transform(img)
      assert(img.shape == (3, 376, 1242))
    else :
      assert(img.shape == (376, 1242, 3))
    
    if self.split == 'train':
      return img, mask
    else :
      return img
  def encode_segmap(self, mask):
    '''
    Sets void classes to zero so they won't be considered for training
    '''
    for voidc in self.void_labels :
      mask[mask == voidc] = self.ignore_index
    for validc in self.valid_labels :
      mask[mask == validc] = self.class_map[validc]
    return mask

  def get_filenames(self, path):
    files_list = list()
    for filename in os.listdir(path):
      files_list.append(os.path.join(path, filename))
    return files_list

In [9]:
class Segmenataion_Model(pl.LightningModule):
  def __init__(self):
    super(Segmenataion_Model,self).__init__()
    self.batch_size=4
    self.lr=1e-3
    self.net=UNet(num_classes=19)
    self.transform=transforms.Compose([transforms.ToTensor(),transforms.Normalize(mean = [0.35675976, 0.37380189, 0.3764753], std = [0.32064945, 0.32098866, 0.32325324])])
    self.training_set=sem_data_creator(split = 'train', transform = self.transform)
    self.testing_set=sem_data_creator(split = 'train', transform = self.transform)
  def forward(self,x):
    return self.net(x)
  def training_step(self, batch,batch_nb):
    # print(batch.size())
    img,mask=batch
    img=img.float()
    mask=mask.long()
    output=self.forward(img)
    loss=F.cross_entropy(output,mask,ignore_index=250)
    return {'loss':loss}
  def configure_optimizers(self):
    opt = torch.optim.Adam(self.net.parameters(), lr = self.lr)
    sch = torch.optim.lr_scheduler.CosineAnnealingLR(opt, T_max = 10)
    return [opt], [sch]

  def train_dataloader(self):
    return DataLoader(self.training_set, batch_size = self.batch_size, shuffle = True)
  def test_dataloader(self):
    return DataLoader(self.testing_set, batch_size = self.batch_size, shuffle = True)
  

In [10]:
model = Segmenataion_Model()
checkpoint_callback = pl.callbacks.ModelCheckpoint(dirpath = '/content/gdrive/MyDrive/data_semantics/checkpoints_unet/',verbose = True, 
    monitor = 'loss',
    mode = 'min')
trainer = pl.Trainer(gpus = 1, max_epochs = 70, callbacks = checkpoint_callback)
trainer.fit(model)

/usr/local/lib/python3.8/dist-packages/pytorch_lightning/trainer/connectors/accelerator_connector.py:441: LightningDeprecationWarning: Setting `Trainer(gpus=1)` is deprecated in v1.7 and will be removed in v2.0. Please use `Trainer(accelerator='gpu', devices=1)` instead.
  rank_zero_deprecation(
INFO:pytorch_lightning.utilities.rank_zero:GPU available: True (cuda), used: True
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:IPU available: False, using: 0 IPUs
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs
INFO:pytorch_lightning.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
INFO:pytorch_lightning.callbacks.model_summary:
  | Name | Type | Params
------------------------------
0 | net  | UNet | 31.0 M
------------------------------
31.0 M    Trainable params
0         Non-trainable params
31.0 M    Total params
124.179   Total estimated model params size (MB)


Training: 0it [00:00, ?it/s]

/usr/local/lib/python3.8/dist-packages/pytorch_lightning/callbacks/model_checkpoint.py:368: UserWarning: `ModelCheckpoint(monitor='loss')` could not find the monitored key in the returned metrics: ['epoch', 'step']. HINT: Did you call `log('loss', value)` in the `LightningModule`?
  warning_cache.warn(m)
INFO:pytorch_lightning.utilities.rank_zero:Epoch 0, global step 50: 'loss' was not in top 1
INFO:pytorch_lightning.utilities.rank_zero:Epoch 1, global step 100: 'loss' was not in top 1
INFO:pytorch_lightning.utilities.rank_zero:Epoch 2, global step 150: 'loss' was not in top 1
INFO:pytorch_lightning.utilities.rank_zero:Epoch 3, global step 200: 'loss' was not in top 1
INFO:pytorch_lightning.utilities.rank_zero:Epoch 4, global step 250: 'loss' was not in top 1
INFO:pytorch_lightning.utilities.rank_zero:Epoch 5, global step 300: 'loss' was not in top 1
INFO:pytorch_lightning.utilities.rank_zero:Epoch 6, global step 350: 'loss' was not in top 1
INFO:pytorch_lightning.utilities.rank_zero:E